# Hierarchical Indexing

We often need to handle higher-dimensional data–that is, data indexed by more than one or two keys.

Pandas provides ``Panel`` and ``Panel4D`` objects that natively handle 3D & 4D data, but it's more common to use __hierarchical indexing__ to use multiple index levels within a single index.

This allows higher-dimensional data can be compactly represented within the familiar 1D ``Series`` and 2D ``DataFrame`` objects.

In [10]:
import pandas as pd
import numpy as np

### Multiply-Indexed Series

Example: how to represent 2D data in a 1D ``Series``? We'll use a dataset where each point has a character and numerical key.

(The bad way.) You might be tempted to use Python tuples as keys:

In [11]:
indexes = [('California', 2000), ('California', 2010),
         ('New York',   2000), ('New York',   2010),
         ('Texas',      2000), ('Texas',      2010)]

populations = [33871648, 37253956,
               18976457, 19378102,
               20851820, 25145561]

pop = pd.Series(populations, index=indexes)
pop

(California, 2000)    33871648
(California, 2010)    37253956
(New York, 2000)      18976457
(New York, 2010)      19378102
(Texas, 2000)         20851820
(Texas, 2010)         25145561
dtype: int64

You can index or slice the series based on this multiple index. But __if you need to select all values from 2010, you need to do some messy munging to make it happen.__

In [12]:
pop[('California', 2010):('Texas', 2000)]

(California, 2010)    37253956
(New York, 2000)      18976457
(New York, 2010)      19378102
(Texas, 2000)         20851820
dtype: int64

In [13]:
pop[[i for i in pop.index if i[1] == 2010]]

(California, 2010)    37253956
(New York, 2010)      19378102
(Texas, 2010)         25145561
dtype: int64

### A Better Way: Pandas MultiIndex

Tuple-based indexing is a rudimentary multi-index. The Pandas ``MultiIndex`` type gives us the type of operations we need. Create a multi-index from the tuples as follows:

In [14]:
index = pd.MultiIndex.from_tuples(index)
index

MultiIndex(levels=[['California', 'New York', 'Texas'], [2000, 2010]],
           labels=[[0, 0, 1, 1, 2, 2], [0, 1, 0, 1, 0, 1]])

The ``MultiIndex`` contains multiple *levels* of indexing–in this case, the state names and the years, as well as multiple *labels* for each data point which encode these levels.

Let's re-index the series with this ``MultiIndex`` and see the result.

In [15]:
pop = pop.reindex(index)
pop

California  2000    33871648
            2010    37253956
New York    2000    18976457
            2010    19378102
Texas       2000    20851820
            2010    25145561
dtype: int64

__The first two columns of the ``Series`` show the multiple index values__; the third column shows the data.

Some entries are missing in the first column: in this multi-index representation, __any blank entry indicates the same value as the line above it.__

Use Pandas slice notation to access any data where the second index is 2010.

In [16]:
pop[:, 2010]

California    37253956
New York      19378102
Texas         25145561
dtype: int64

### MultiIndex as an extra dimension

We could store the same data using a ``DataFrame`` with index and column labels. Pandas is built with this equivalence in mind. The __``unstack()`` method__ will convert a multiply-indexed ``Series`` into a conventionally indexed ``DataFrame``. __``stack()``__ provides the opposite operation.

In [17]:
pop_df = pop.unstack()
pop_df

,2000,2010
California,33871648,37253956
New York,18976457,19378102
Texas,20851820,25145561


In [19]:
pop_df.stack()
pop_df

,2000,2010
California,33871648,37253956
New York,18976457,19378102
Texas,20851820,25145561


Hierarchical indexing allows us to represent data of three or more dimensions in a ``Series`` or ``DataFrame``. Each extra level in a multi-index represents an extra dimension. 

Now we can add a column of demographic data for each state at each year. With a ``MultiIndex`` this is as easy as adding another column to the ``DataFrame``.

In [20]:
pop_df = pd.DataFrame({'total': pop,
                       'under18': [9267089, 9284094,
                                   4687374, 4318033,
                                   5906301, 6879014]})
pop_df

total  under18
California 2000  33871648  9267089
           2010  37253956  9284094
New York   2000  18976457  4687374
           2010  19378102  4318033
Texas      2000  20851820  5906301
           2010  25145561  6879014

In [21]:
f_u18 = pop_df['under18'] / pop_df['total']
f_u18.unstack()

,2000,2010
California,0.273594,0.249211
New York,0.247010,0.222831
Texas,0.283251,0.273568


### Create a MultiIndex

The simplest way to build a multiply indexed ``Series`` or ``DataFrame`` is to __pass a list of two or more index arrays to the constructor.__

In [23]:
df = pd.DataFrame(np.random.rand(4, 2),
                  index=[
                      ['a', 'a', 'b', 'b'], 
                      [1, 2, 1, 2]],
                  columns=[
                      'data1', 
                      'data2'])
df

data1     data2
a 1  0.933642  0.689267
  2  0.065625  0.979676
b 1  0.429007  0.001207
  2  0.340656  0.924987

If you pass a dictionary with appropriate tuples as keys, __Pandas will recognize this and use a ``MultiIndex`` by default.__

In [24]:
data = {('California', 2000): 33871648,
        ('California', 2010): 37253956,
        ('Texas',      2000): 20851820,
        ('Texas',      2010): 25145561,
        ('New York',   2000): 18976457,
        ('New York',   2010): 19378102}
pd.Series(data)

California  2000    33871648
            2010    37253956
New York    2000    18976457
            2010    19378102
Texas       2000    20851820
            2010    25145561
dtype: int64

### Explicit MultiIndex constructors

We can also __construct a ``MultiIndex`` from a list of arrays__ which specify the index values within each level:

In [25]:
pd.MultiIndex.from_arrays([
    ['a', 'a', 'b', 'b'], 
    [ 1,   2,   1,   2 ]])

MultiIndex(levels=[['a', 'b'], [1, 2]],
           labels=[[0, 0, 1, 1], [0, 1, 0, 1]])

You can also construct it __from a list of tuples in the same manner:__

In [26]:
pd.MultiIndex.from_tuples(
    [('a', 1), ('a', 2), ('b', 1), ('b', 2)])

MultiIndex(levels=[['a', 'b'], [1, 2]],
           labels=[[0, 0, 1, 1], [0, 1, 0, 1]])

You can even construct it __from a Cartesian product of single indices:__

In [27]:
pd.MultiIndex.from_product(
    [['a', 'b'], [1, 2]])

MultiIndex(levels=[['a', 'b'], [1, 2]],
           labels=[[0, 0, 1, 1], [0, 1, 0, 1]])

You can construct a ``MultiIndex`` using its internal encoding by __passing ``levels``__ (a list of lists containing available index values for each level) __and ``labels``__ (a list of lists that reference these labels):

In [28]:
pd.MultiIndex(levels=[['a', 'b'], [1, 2]],
              labels=[[0, 0, 1, 1], [0, 1, 0, 1]])

MultiIndex(levels=[['a', 'b'], [1, 2]],
           labels=[[0, 0, 1, 1], [0, 1, 0, 1]])

Any of these objects can be passed as the ``index`` argument when creating a ``Series`` or ``Dataframe``, or be passed to the ``reindex`` method of an existing ``Series`` or ``DataFrame``.

### MultiIndex level names

Sometimes it helps to name the levels of the ``MultiIndex``. Do this by passing the ``names`` argument to a constructor, or by setting the ``names`` attribute of the index after the fact. With more involved datasets, this can be a useful way to keep track of the meaning of various index values.

In [29]:
pop.index.names = ['state', 'year']
pop

state       year
California  2000    33871648
            2010    37253956
New York    2000    18976457
            2010    19378102
Texas       2000    20851820
            2010    25145561
dtype: int64

### MultiIndex for columns

``DataFrame`` rows and columns are symmetric. Both can have multiple levels of indices.

In [31]:
index   = pd.MultiIndex.from_product([[2013, 2014],         [1, 2]           ], names=['year', 'visit'])
columns = pd.MultiIndex.from_product([['Bob', 'Guido', 'Sue'], ['HR', 'Temp']], names=['subject', 'type'])

# mock some data
data = np.round(np.random.randn(4, 6), 1)
data[:, ::2] *= 10
data += 37

# create the DataFrame
health_data = pd.DataFrame(data, index=index, columns=columns)
health_data

subject      Bob       Guido         Sue      
type          HR  Temp    HR  Temp    HR  Temp
year visit                                    
2013 1      37.0  37.3  33.0  37.1  56.0  36.6
     2      42.0  38.5  39.0  38.5  25.0  37.0
2014 1      50.0  36.9  36.0  36.5  48.0  37.0
     2      34.0  38.0  35.0  36.8  27.0  36.8

This is fundamentally four-dimensional data (subject, measurement type, year, visit number). Now we can index the top-level column by the person's name and get a full ``DataFrame`` containing just that person's information:

In [32]:
health_data['Guido']

type          HR  Temp
year visit            
2013 1      33.0  37.1
     2      39.0  38.5
2014 1      36.0  36.5
     2      35.0  36.8

### Indexing and Slicing a MultiIndex

Indexing and slicing a ``MultiIndex`` is designed to be intuitive. It helps to think of indices as added dimensions.

### Multiply indexed Series

Consider the multiply indexed ``Series`` of state populations we saw earlier - we can access single elements using multiple-term indexes.

In [33]:
pop

state       year
California  2000    33871648
            2010    37253956
New York    2000    18976457
            2010    19378102
Texas       2000    20851820
            2010    25145561
dtype: int64

In [34]:
pop['California', 2000]

33871648

``MultiIndex`` __supports partial indexing__ (indexing just one of the levels in the index). The result is another ``Series``, with the lower-level indices maintained.


In [35]:
pop['California']

year
2000    33871648
2010    37253956
dtype: int64

Partial slicing is supported as long as the ``MultiIndex`` is sorted.

In [36]:
pop.loc['California':'New York']

state       year
California  2000    33871648
            2010    37253956
New York    2000    18976457
            2010    19378102
dtype: int64

Partial indexing can be performed on lower levels __by passing an empty slice in the first index:__

In [37]:
pop[:, 2000]

state
California    33871648
New York      18976457
Texas         20851820
dtype: int64

Selection based on __Boolean masks works too:__

In [38]:
pop[pop > 22000000]

state       year
California  2000    33871648
            2010    37253956
Texas       2010    25145561
dtype: int64

Selection based on __fancy indexing works too:__

In [39]:
pop[['California', 'Texas']]

state       year
California  2000    33871648
            2010    37253956
Texas       2000    20851820
            2010    25145561
dtype: int64

### Multiply indexed DataFrames

A multiply indexed ``DataFrame`` behaves in a similar manner. Consider our toy medical ``DataFrame`` from before:

In [40]:
health_data

subject      Bob       Guido         Sue      
type          HR  Temp    HR  Temp    HR  Temp
year visit                                    
2013 1      37.0  37.3  33.0  37.1  56.0  36.6
     2      42.0  38.5  39.0  38.5  25.0  37.0
2014 1      50.0  36.9  36.0  36.5  48.0  37.0
     2      34.0  38.0  35.0  36.8  27.0  36.8

__Columns are primary in a ``DataFrame``__. The syntax used for multiply indexed ``Series`` applies to the columns. For example, we can recover Guido's heart rate data with a simple operation:

In [41]:
health_data['Guido', 'HR']

year  visit
2013  1        33.0
      2        39.0
2014  1        36.0
      2        35.0
Name: (Guido, HR), dtype: float64

We can use the ``loc``, ``iloc``, and ``ix`` index methods.

In [42]:
health_data.iloc[:2, :2]

subject      Bob      
type          HR  Temp
year visit            
2013 1      37.0  37.3
     2      42.0  38.5

Indexers provide an array-like view of the underlying data, but __each index in ``loc`` or ``iloc`` can be passed a tuple of multiple indices:__

In [43]:
health_data.loc[:, ('Bob', 'HR')]

year  visit
2013  1        37.0
      2        42.0
2014  1        50.0
      2        34.0
Name: (Bob, HR), dtype: float64

Slices within index tuples is not convenient; __trying to create a slice within a tuple will cause a syntax error.__
You can solve this by building a slice with Python's built-in slice() function, but __using a Pandas IndexSlice is better.__

In [44]:
health_data.loc[(:, 1), (:, 'HR')]

SyntaxError: invalid syntax (<ipython-input-44-fb34fa30ac09>, line 1)

In [45]:
idx = pd.IndexSlice
health_data.loc[idx[:, 1], idx[:, 'HR']]

,subject,Bob,Guido,Sue
,type,HR,HR,HR
year,visit,,,
2013,1,37.0,33.0,56.0
2014,1,50.0,36.0,48.0


### Rearranging Multi-Indices

One of the keys to working with multiply indexed data is knowing how to effectively transform the data. There are several commands that will preserve all the information in a dataset, but rearrange it for subsequent operations. We've already seen ``stack()`` and ``unstack()`` methods. Here are some additional ones.

### Sorted and unsorted indices

Recall that __many ``MultiIndex`` slicing operations will fail if the index is not sorted.__ We'll start by creating some simple multiply indexed data where the indices are *not lexographically sorted*:

In [46]:
index = pd.MultiIndex.from_product([['a', 'c', 'b'], [1, 2]])
data = pd.Series(np.random.rand(6), index=index)
data.index.names = ['char', 'int']
data

char  int
a     1      0.632029
      2      0.929278
c     1      0.156645
      2      0.657699
b     1      0.757224
      2      0.658689
dtype: float64

A partial slice of this index returns an error:

In [47]:
try:
    data['a':'b']
except KeyError as e:
    print(type(e))
    print(e)

<class 'pandas.errors.UnsortedIndexError'>
'Key length (1) was greater than MultiIndex lexsort depth (0)'


Although it is not clear from the error message, __this is the result of the MultiIndex not being sorted.__  For various reasons, partial slices and similar operations require the levels in the ``MultiIndex`` to be in sorted order. Pandas provides several sorting routines including ``sort_index()`` and ``sortlevel()``. We'll use the simplest, ``sort_index()``, here:

In [48]:
data = data.sort_index()
data

char  int
a     1      0.632029
      2      0.929278
b     1      0.757224
      2      0.658689
c     1      0.156645
      2      0.657699
dtype: float64

Now partial slicing will work:

In [49]:
data['a':'b']

char  int
a     1      0.632029
      2      0.929278
b     1      0.757224
      2      0.658689
dtype: float64

### Stacking and unstacking indices

We can convert a dataset from a stacked multi-index to a simple 2D representation, optionally specifying the level:

In [50]:
pop.unstack(level=0)

state,California,New York,Texas
year,,,
2000,33871648,18976457,20851820
2010,37253956,19378102,25145561


In [51]:
pop.unstack(level=1)

year,2000,2010
state,,
California,33871648,37253956
New York,18976457,19378102
Texas,20851820,25145561


The opposite of ``unstack()`` is ``stack()``, which can be used to recover the original series:

In [52]:
pop.unstack().stack()

state       year
California  2000    33871648
            2010    37253956
New York    2000    18976457
            2010    19378102
Texas       2000    20851820
            2010    25145561
dtype: int64

### Index setting and resetting

Another way to rearrange hierarchical data is to convert index labels into columns using the ``reset_index`` method. Calling this on the population dictionary will result in a ``DataFrame`` with a *state* and *year* column holding the information that was formerly in the index. For clarity, we can optionally specify the name of the data for the column representation:

In [53]:
pop_flat = pop.reset_index(name='population')
pop_flat

,state,year,population
0,California,2000,33871648
1,California,2010,37253956
2,New York,2000,18976457
3,New York,2010,19378102
4,Texas,2000,20851820
5,Texas,2010,25145561


Raw real-world data often looks like this - it's useful to build a ``MultiIndex`` from the column values. __This can be done with the ``set_index`` method__ of the ``DataFrame``, which returns a multiply indexed ``DataFrame``:

In [54]:
pop_flat.set_index(['state', 'year'])

population
state      year            
California 2000    33871648
           2010    37253956
New York   2000    18976457
           2010    19378102
Texas      2000    20851820
           2010    25145561

### Data Aggregations on Multi-Indices

Pandas has built-in data aggregation methods, such as ``mean()``, ``sum()``, and ``max()``. Hierarchically indexed datasets can accept a ``level`` parameter that controls which subset of the data the aggregate affects.

In [55]:
health_data

subject      Bob       Guido         Sue      
type          HR  Temp    HR  Temp    HR  Temp
year visit                                    
2013 1      37.0  37.3  33.0  37.1  56.0  36.6
     2      42.0  38.5  39.0  38.5  25.0  37.0
2014 1      50.0  36.9  36.0  36.5  48.0  37.0
     2      34.0  38.0  35.0  36.8  27.0  36.8

Perhaps we'd like to average out the measurements in the two visits each year. Do this by naming the index level we'd like to explore, in this case the year:

In [56]:
data_mean = health_data.mean(level='year')
data_mean

subject   Bob        Guido          Sue      
type       HR   Temp    HR   Temp    HR  Temp
year                                         
2013     39.5  37.90  36.0  37.80  40.5  36.8
2014     42.0  37.45  35.5  36.65  37.5  36.9

By further making use of the ``axis`` keyword, we can take the mean among levels on the columns as well:

In [57]:
data_mean.mean(axis=1, level='type')

type,HR,Temp
year,,
2013,38.666667,37.5
2014,38.333333,37.0
